In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
np.random.seed(0)

In [2]:
testOriginal = pd.read_csv("Arquivos csv/test.csv")
trainOriginal = pd.read_csv("Arquivos csv/train.csv")

In [3]:
# Exercício 8.1
# Vou usar a manipulação feita no exercício do cap07 para tratar de valores em falta
test = testOriginal.drop(['Cabin', 'Ticket'], axis = 1)
train = trainOriginal.drop(['Cabin', 'Ticket'], axis = 1)

test['Age'] = test['Age'].fillna(test['Age'].median())
train['Age'] = train['Age'].fillna(train['Age'].median())

test['Fare'] = test['Fare'].fillna(test['Fare'].median())
train['Fare'] = train['Fare'].fillna(train['Fare'].median())

test['Embarked'] = test['Embarked'].fillna(test['Embarked'].mode()[0])
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])

# Name e PassengerId não é uma informação que influencia se a pessoa vai sobreviver,
# logo será apagada
train = train.drop(['Name', 'PassengerId'], axis = 1)
test = test.drop(['Name', 'PassengerId'], axis = 1)

# Questão 01
print("Informações do banco de dados 'train.csv'\n")
print(train.info())

# Obter lista de variáveis categóricas
object_cols = list(train.select_dtypes(include = object).columns)

print("\n Variáveis categóricas: \n", object_cols)

# Obter lista de variáveis numéricas
num_cols = list(train.select_dtypes(include = [int, float]).columns)

print("\n Variáveis numéricas: \n", num_cols)

# Função que realiza one-hot encode
def ordinalEncoding(df):
    dfCopia = df
    OH_encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
    OH_cols = pd.DataFrame(OH_encoder.fit_transform(dfCopia[object_cols]))
    OH_cols.index = df.index
    OH_cols.columns = OH_encoder.get_feature_names_out(object_cols)
    dfCopia = dfCopia.drop(object_cols, axis = 1)
    OH_X = pd.concat([dfCopia, OH_cols], axis=1)
    return OH_X

newTrain = ordinalEncoding(train)
newTest = ordinalEncoding(test)

# Questão 2
# Criar variável X, contendo as Features
X = newTrain.drop('Survived', axis = 1)

# Criar variável y, contendo o target
y = newTrain['Survived']

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0, test_size = 0.2)

# Treinar random forest com as features e o target de treino
modelo_titanic = RandomForestClassifier()

# Realizar a predição
modelo_titanic.fit(train_X, train_y)

predicted_survived = modelo_titanic.predict(val_X)
qualidade = accuracy_score(val_y, predicted_survived)
print("\nPorcentagem das predições que foram acertadas pelo modelo: {:.2f}%".format((qualidade*100)))

Informações do banco de dados 'train.csv'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB
None

 Variáveis categóricas: 
 ['Sex', 'Embarked']

 Variáveis numéricas: 
 ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

Porcentagem das predições que foram acertadas pelo modelo: 82.68%


In [4]:
# Exercício 8.2
# Questão 01
test = testOriginal.drop(['Cabin', 'Ticket', 'Name', 'PassengerId'], axis = 1)
train = trainOriginal.drop(['Cabin', 'Ticket', 'Name', 'PassengerId'], axis = 1)

test['Fare'] = test['Fare'].fillna(test['Fare'].median())
train['Fare'] = train['Fare'].fillna(train['Fare'].median())

# Questao 2
def createPipelines(data, encoder, model, numerical_imputer, categorical_imputer):
    """"Essa função produz e avalia pipelines que utilizam estratégias indicadas
    nos argumentos passados e, como saída, retorna o resultado da avaliação em porcentagem"""
    # Definindo as features e o target que serão utilizados
    X = data.drop(['Survived'], axis = 1)
    y = data['Survived']
    
    train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0, test_size = 0.2)
    
    # Pré-processamento de dados numéricos
    numerical_transformer = numerical_imputer
    
    # Pré-processamento de dados categóricos
    categorical_transformer = Pipeline(steps = [('imputer', categorical_imputer), 
                                                ('encoder', encoder)])
    
    # Pré-processamento de pacotes para dados numéricos e categóricos
    categorical_cols = list(X.select_dtypes(include = object).columns)
    numerical_cols = list(X.select_dtypes(include = [int, float]).columns)
    preprocessor = ColumnTransformer(transformers = [('num', numerical_transformer, numerical_cols),
                                                     ('cat', categorical_transformer, categorical_cols)])
    
    # Pré-processamento e modelagem em um pipeline
    my_pipeline = Pipeline(steps = [('preprocessor', preprocessor),
                                    ('model', model)])
    
    # Pré-processamento de dados de treinamento, modo de ajuste
    my_pipeline.fit(train_X, train_y)
    
    # Pré-processamento de dados de validação, obter previsões
    preds = my_pipeline.predict(val_X)
    
    # Avalie o modelo
    score = accuracy_score(val_y, preds)
    
    return (score * 100)

score = createPipelines(train, 
                        OneHotEncoder(handle_unknown = 'ignore'), 
                        RandomForestClassifier(), 
                        SimpleImputer(strategy = 'median'),
                        SimpleImputer(strategy = 'most_frequent'))

print("\nPorcentagem das predições que foram acertadas pelo modelo Pipelines é: {:.2f}%".format((score)))

# Questao 3
imputers = ['mean', 'most_frequent', 'median', 'constant']
encoders = [OneHotEncoder(handle_unknown = 'ignore'), OrdinalEncoder()]

# Questão 4
for encoder in encoders:
    for imputer in imputers:
        score = createPipelines(train, 
                                encoder, 
                                RandomForestClassifier(), 
                                SimpleImputer(strategy = imputer),
                                SimpleImputer(strategy = 'most_frequent'))
        print("\nPorcentagem das predições que foram acertadas pelo modelo usando o encoder '{}' e o imputer '{}': {:.2f}%".format(encoder, imputer, score))
        
# Questão 5
print("\nA precissão de todos os modelos utilizados nos exercícios do capítulo 8.1 e 8.2 estão bem próximas")


Porcentagem das predições que foram acertadas pelo modelo Pipelines é: 84.92%

Porcentagem das predições que foram acertadas pelo modelo usando o encoder 'OneHotEncoder(handle_unknown='ignore')' e o imputer 'mean': 84.36%

Porcentagem das predições que foram acertadas pelo modelo usando o encoder 'OneHotEncoder(handle_unknown='ignore')' e o imputer 'most_frequent': 84.92%

Porcentagem das predições que foram acertadas pelo modelo usando o encoder 'OneHotEncoder(handle_unknown='ignore')' e o imputer 'median': 83.80%

Porcentagem das predições que foram acertadas pelo modelo usando o encoder 'OneHotEncoder(handle_unknown='ignore')' e o imputer 'constant': 84.36%

Porcentagem das predições que foram acertadas pelo modelo usando o encoder 'OrdinalEncoder()' e o imputer 'mean': 83.24%

Porcentagem das predições que foram acertadas pelo modelo usando o encoder 'OrdinalEncoder()' e o imputer 'most_frequent': 83.24%

Porcentagem das predições que foram acertadas pelo modelo usando o encoder '

In [5]:
# Exercicio 8.3
# Questão 1 - O modelo que se saiu melhor no teste foi:
# encoder: OneHotEncoder e o imputer = 'most_frequent' 

# Questão 2
#Letra c
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0, 
                                                  test_size = 0.2)

my_model = GradientBoostingClassifier(random_state = (0), n_iter_no_change = (100))
my_model.fit(train_X, train_y)

predictions = my_model.predict(val_X)
scores = cross_val_score(my_model, X, y, cv = 5, scoring = 'accuracy')

print("\nPorcentagem das predições que foram acertadas pelo modelo Cross validation e Gradient Boosting foi: {:.2f}%".format(scores.mean()*100))

# Letra d
print("\nO algoritimo que se saiu melhor foi o Gradient Boosting")


Porcentagem das predições que foram acertadas pelo modelo Cross validation e Gradient Boosting foi: 83.84%

O algoritimo que se saiu melhor foi o Gradient Boosting
